# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load in cities data from WeatherPy
cities_data = pd.read_csv("../output_data/cities.csv")
cities_df = pd.DataFrame(cities_data)
cities_df.head()

,Unnamed: 0,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,{0},Crib Point,-38.3500,145.2000,86.0,48,0,4.23,AU,2021-02-10 02:29:49
1,1,{1},Barrow,71.2906,-156.7887,-22.0,69,1,17.27,US,2021-02-10 02:24:57
2,2,{2},Lata,40.1629,-8.3327,55.0,93,100,7.00,PT,2021-02-10 02:29:49
3,3,{3},Progreso,21.2833,-89.6667,77.0,78,40,14.97,MX,2021-02-10 02:29:49
4,4,{5},Ushuaia,-54.8000,-68.3000,62.6,72,0,4.61,AR,2021-02-10 02:29:50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [4]:
# Create the locations and their weights based on humidity
locations = cities_df[["Lat", "Lng"]].astype(float)
city_weights = cities_df["Humidity"].astype(float)

In [5]:
# Create a preliminary heatmap
fig = gmaps.figure(center=(30,20), zoom_level=1.5)

heat_layer = gmaps.heatmap_layer(locations, weights=city_weights, dissipating=False, max_intensity=city_weights.max(),
                                point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Create the ideal dataframe for my ice fishing trip
hotel_df = pd.DataFrame()
hotel_df = cities_df.loc[(cities_df["Max Temp"] < 25) & (cities_df["Wind Speed"] > 12) & (cities_df["Cloudiness"] > 50)]

hotel_df

,Unnamed: 0,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,{10},Ust'-Omchug,61.1500,149.6333,4.71,90,100,12.88,RU,2021-02-10 02:29:50
81,81,{93},Berlevåg,70.8578,29.0864,21.97,88,100,19.28,NO,2021-02-10 02:29:59
179,179,{196},Harboøre,56.6175,8.1807,23.00,87,100,18.32,DK,2021-02-10 02:30:23
297,297,{324},Yar-Sale,66.8333,70.8333,-20.51,85,93,13.96,RU,2021-02-10 02:30:44
350,350,{380},Ostrovnoy,68.0531,39.5131,6.55,93,90,17.02,RU,2021-02-10 02:30:52
402,402,{436},Mehamn,71.0357,27.8492,20.79,89,100,19.51,NO,2021-02-10 02:31:01
457,457,{495},Talaya,61.3833,152.7500,2.21,91,100,14.38,RU,2021-02-10 02:31:10
478,478,{519},Evensk,61.9500,159.2333,15.22,89,100,25.37,RU,2021-02-10 02:31:13
479,479,{520},Slave Lake,55.2834,-114.7690,-18.40,69,90,21.85,CA,2021-02-10 02:31:13


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Hotel name column creation
hotel_df["Hotel Name"] = ""

C:\Users\NPH0420\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
# Create the base url for the api search 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key
}

In [9]:
# Call the google api for each city and find the nearest hotel within 5000 meters
for index, row in hotel_df.iterrows():
    
    # Grab the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Assign the location of the city (lat and lng)
    city_coord = f"{lat}, {lng}"
    
    # Add location params to the api call
    params["location"] = city_coord
    
    # Make the api call
    response = requests.get(base_url, params=params).json()
    
    # Getting responses into the results section of the api     
    results = response['results']
     
    # Account for exceptions
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        print(f'The nearest hotel to {row["City"]} is {results[0]["name"]}')
        
    except:
        print("Could not find a hotel within range")
    
    #Ensure calls do not time out or exceed number of allowed calls
    time.sleep(1)

Could not find a hotel within range


C:\Users\NPH0420\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


The nearest hotel to Berlevåg is Berlevåg camping & Appartement AS
The nearest hotel to Harboøre is Harboøre Hotel
The nearest hotel to Yar-Sale is Yalemd
Could not find a hotel within range
The nearest hotel to Mehamn is Mehamn Arctic Hotel AS
Could not find a hotel within range
Could not find a hotel within range
The nearest hotel to Slave Lake is Lakeview Inns & Suites - Slave Lake


In [10]:
hotel_df

,Unnamed: 0,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
9,9,{10},Ust'-Omchug,61.1500,149.6333,4.71,90,100,12.88,RU,2021-02-10 02:29:50,
81,81,{93},Berlevåg,70.8578,29.0864,21.97,88,100,19.28,NO,2021-02-10 02:29:59,Berlevåg camping & Appartement AS
179,179,{196},Harboøre,56.6175,8.1807,23.00,87,100,18.32,DK,2021-02-10 02:30:23,Harboøre Hotel
297,297,{324},Yar-Sale,66.8333,70.8333,-20.51,85,93,13.96,RU,2021-02-10 02:30:44,Yalemd
350,350,{380},Ostrovnoy,68.0531,39.5131,6.55,93,90,17.02,RU,2021-02-10 02:30:52,
402,402,{436},Mehamn,71.0357,27.8492,20.79,89,100,19.51,NO,2021-02-10 02:31:01,Mehamn Arctic Hotel AS
457,457,{495},Talaya,61.3833,152.7500,2.21,91,100,14.38,RU,2021-02-10 02:31:10,
478,478,{519},Evensk,61.9500,159.2333,15.22,89,100,25.37,RU,2021-02-10 02:31:13,
479,479,{520},Slave Lake,55.2834,-114.7690,-18.40,69,90,21.85,CA,2021-02-10 02:31:13,Lakeview Inns & Suites - Slave Lake


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))